In [1]:
import warnings
warnings.filterwarnings("ignore")
# !pip install pycocotools --user

In [2]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
import glob
import matplotlib.pyplot as plt
from keras.models import load_model
import ntpath
from sklearn.metrics import confusion_matrix

In [3]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
valid_path = '../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

# **Using InceptionV3 as trained on ImageNet Without finetunig**

In [4]:
# Import the InceptionV3 library as shown below and add preprocessing layer to the front of InceptionV3
# Here we will be using imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [5]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [6]:
# useful for getting number of output classes
folders = glob.glob('../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/*')
print(len(folders))

0


In [7]:
print("Building model with InceptionV3 with imagenet weights")
model = Sequential([
    inception,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(rate=0.2),
    Dense(len(folders), activation='softmax')
])


model.summary()

Building model with InceptionV3 with imagenet weights
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 51200)             0         
                                                                 
 dense (Dense)               (None, 512)               26214912  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 0)                 0         
                                                                 
Total params: 48017696 (183.17 MB)
Trainable params: 26214912 (100.00 MB)
Non-trainable params: 21802784 (83.17 MB)
__________________

In [8]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [9]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [11]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('E:/projects/Crop_P_rediction/Crop_Prediction/crop_prediction_map/dataset/eaf-disease-detection',
                                                 target_size = (224, 224),
                                                 batch_size = 50,
                                                 class_mode = 'categorical')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\jiten\AppData\Local\Temp\ipykernel_16920\2435246195.py", line 2, in <module>
    training_set = train_datagen.flow_from_directory('E:/projects/Crop_P_rediction/Crop_Prediction/crop_prediction_map/dataset/eaf-disease-detection',
  File "c:\Users\jiten\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\preprocessing\image.py", line 1649, in flow_from_directory
    return DirectoryIterator(
  File "c:\Users\jiten\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\preprocessing\image.py", line 563, in __init__
    for subdir in sorted(os.listdir(directory)):
FileNotFoundError: [WinError 3] The system cannot find the path specified: 'E:/projects/Crop_P_rediction/Crop_Prediction/crop_prediction_map/dataset/eaf-disease-detection'

In [13]:
test_set = test_datagen.flow_from_directory('..E:/projects/Crop_P_rediction/Crop_Prediction/crop_prediction_map/dataset/leaf-disease-detection.ipynb',
                                            target_size = (224, 224),
                                            batch_size = 50,
                                            class_mode = 'categorical')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\jiten\AppData\Local\Temp\ipykernel_16920\1955179718.py", line 1, in <module>
    test_set = test_datagen.flow_from_directory('..E:/projects/Crop_P_rediction/Crop_Prediction/crop_prediction_map/dataset/leaf-disease-detection.ipynb',
  File "c:\Users\jiten\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\preprocessing\image.py", line 1649, in flow_from_directory
    return DirectoryIterator(
  File "c:\Users\jiten\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\preprocessing\image.py", line 563, in __init__
    for subdir in sorted(os.listdir(directory)):
OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: '..E:/projects/Crop_P_rediction/Crop_Prediction/crop_prediction_map/dataset/

In [12]:
history = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\jiten\AppData\Local\Temp\ipykernel_11168\1748877077.py", line 2, in <module>
    training_set,
NameError: name 'training_set' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1428, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1319, in structured_traceback
    return VerboseTB.structur

In [13]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']
EPOCHS=10

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\jiten\AppData\Local\Temp\ipykernel_11168\1321096600.py", line 1, in <module>
    acc = history.history['accuracy']
NameError: name 'history' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1428, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1319, in structured_traceback
    return Ver

# **Using InceptionV3 while fine tuning the top 2 blocks**

In [14]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in inception.layers[:249]:
    layer.trainable = False
for layer in inception.layers[249:]:
    layer.trainable = True

In [15]:
print("Building model with InceptionV3 with imagenet weights")
model_finetuned = Sequential([
    inception,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(rate=0.2),
    Dense(38, activation='softmax')
])

# tell the model what cost and optimization method to use
model_finetuned.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


model.summary()

Building model with InceptionV3 with imagenet weights
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 51200)             0         
                                                                 
 dense (Dense)               (None, 512)               26214912  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 0)                 0         
                                                                 
Total params: 48017696 (183.17 MB)
Trainable params: 37329792 (142.40 MB)
Non-trainable params: 10687904 (40.77 MB)
__________________

In [16]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import pickle

filepath = "model_finetuned_{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="acc", mode="max", patience=15)

callbacks_list = [checkpoint1, early] #early

history = model_finetuned.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set), 
    callbacks=callbacks_list
)

with open('trainHistoryDict.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)
    

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\jiten\AppData\Local\Temp\ipykernel_11168\2506619901.py", line 11, in <module>
    training_set,
NameError: name 'training_set' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1428, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1319, in structured_traceback
    return VerboseTB.structu

In [17]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']
EPOCHS=10

epochs_range = range(EPOCHS)

plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\jiten\AppData\Local\Temp\ipykernel_11168\2480474801.py", line 1, in <module>
    acc = history.history['accuracy']
NameError: name 'history' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1428, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1319, in structured_traceback
    return Ver

In [18]:
best_model_finetuned = load_model('./model_finetuned_08-0.98.h5')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\jiten\AppData\Local\Temp\ipykernel_11168\1313038484.py", line 1, in <module>
    best_model_finetuned = load_model('./model_finetuned_08-0.98.h5')
  File "c:\Users\jiten\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\saving_api.py", line 262, in load_model
    return legacy_sm_saving_lib.load_model(
  File "c:\Users\jiten\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "c:\Users\jiten\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\saving\legacy\save.py", line 234, in load_model
    raise IOError(
OSError: No file or directory found at ./model_finetuned_08-0.98.h5

During handling 

In [19]:
# Import OpenCV
import cv2

# Utility
import itertools
import random
from collections import Counter
from glob import iglob

test_dir='../input/new-plant-diseases-dataset/test'

def load_image(filename):
    img = cv2.imread(filename)
    img = cv2.resize(img, (IMAGE_SIZE[0], IMAGE_SIZE[1]) )
    img = img /255
    
    return img

classes=['Apple_scab', 'Apple_Black_rot', 'Cedar_apple_rust', 
         'Apple_healthy', 'Blueberry_healthy', 
         'Cherry_Powdery_mildew', 'Cherry_healthy', 
         'Corn_Cercospora_leaf_spot', 'Corn_Common_rust_', 
         'Corn_Northern_Leaf_Blight', 'Corn_healthy', 
         'Grape_Black_rot', 'Grape_Black_Measles', 
         'Grape_Leaf_blight', 'Grape_healthy', 
         'Orange_Haunglongbing', 'Peach_Bacterial_spot', 
         'Peach_healthy', 'Pepper,_bell_Bacterial_spot', 'Pepper,_bell_healthy', 
         'Potato_Early_blight', 'Potato_Late_blight', 'Potato_healthy', 
         'Raspberry_healthy', 'Soybean_healthy', 'Squash_Powdery_mildew', 
         'Strawberry_Leaf_scorch', 'Strawberry_healthy', 'Tomato_Bacterial_spot', 
         'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 
         'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites Two-spotted_spider_mite', 
         'Tomato_Target_Spot', 'Tomato_Tomato_Yellow_Leaf_Curl_Virus', 
         'Tomato_Tomato_mosaic_virus', 'Tomato_healthy']

def predict(image):
    probabilities = best_model_finetuned.predict(np.asarray([img]))[0]
    class_idx = np.argmax(probabilities)
    
    return {classes[class_idx]: probabilities[class_idx]}

In [20]:
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(20,20),text_size=15, norm=False, savefig=True): 
    # Create the confustion matrix
    cm = confusion_matrix(y_true, y_pred)
    cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
    n_classes = cm.shape[0] # find the number of classes we're dealing with
    
    # Plot the figure and make it pretty
    fig, ax = plt.subplots(figsize=figsize)
    cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
    fig.colorbar(cax)
    
    # Are there a list of classes?
    if classes:
        labels = classes
    else:
        labels = np.arange(cm.shape[0])
            
    # Label the axes
    ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes), 
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)
    
    # Make x-axis labels appear on bottom
    ax.xaxis.set_label_position("bottom")
    ax.xaxis.tick_bottom()
    
    ### Added: Rotate xticks for readability & increase font size (required due to such a large confusion matrix)
    plt.xticks(rotation=70, fontsize=text_size)
    plt.yticks(fontsize=text_size)
    
    # Set the threshold for different colors
    threshold = (cm.max() + cm.min()) / 2.
    
    # Plot the text on each cell
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if norm:
            plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
        else:
            plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
            
    # Save the figure to the current working directory
    if savefig:
        fig.savefig("confusion_matrix.png")

In [21]:
Y_pred = best_model_finetuned.predict(test_set, num_of_test_samples // 50 + 1)
y_pred = np.argmax(Y_pred, axis=1)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\jiten\AppData\Local\Temp\ipykernel_11168\824556713.py", line 1, in <module>
    Y_pred = best_model_finetuned.predict(test_set, num_of_test_samples // 50 + 1)
NameError: name 'best_model_finetuned' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1428, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultra

In [22]:
num_of_test_samples=17572
classes=['Apple_scab', 'Apple_Black_rot', 'Cedar_apple_rust', 
         'Apple_healthy', 'Blueberry_healthy', 
         'Cherry_Powdery_mildew', 'Cherry_healthy', 
         'Corn_Cercospora_leaf_spot', 'Corn_Common_rust_', 
         'Corn_Northern_Leaf_Blight', 'Corn_healthy', 
         'Grape_Black_rot', 'Grape_Black_Measles', 
         'Grape_Leaf_blight', 'Grape_healthy', 
         'Orange_Haunglongbing', 'Peach_Bacterial_spot', 
         'Peach_healthy', 'Pepper,_bell_Bacterial_spot', 'Pepper,_bell_healthy', 
         'Potato_Early_blight', 'Potato_Late_blight', 'Potato_healthy', 
         'Raspberry_healthy', 'Soybean_healthy', 'Squash_Powdery_mildew', 
         'Strawberry_Leaf_scorch', 'Strawberry_healthy', 'Tomato_Bacterial_spot', 
         'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 
         'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites', 
         'Tomato_Target_Spot', 'Tomato_Yellow_Leaf_Curl_Virus', 
         'Tomato_mosaic_virus', 'Tomato_healthy']
make_confusion_matrix(test_set.classes, y_pred,classes=classes)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\jiten\AppData\Local\Temp\ipykernel_11168\1604700651.py", line 18, in <module>
    make_confusion_matrix(test_set.classes, y_pred,classes=classes)
NameError: name 'test_set' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1428, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1319, in str

In [23]:
path ='../input/new-plant-diseases-dataset/test/test/'
PList=glob.glob('../input/new-plant-diseases-dataset/test/test/*')
for filename in PList:
    img = load_image(str(filename))
    prediction = predict(img)
    print("ACTUAL CLASS: %s, PREDICTED: class: %s, confidence: %f" % (os.path.basename(filename), list(prediction.keys())[0], list(prediction.values())[0]))
    plt.imshow(img)
    plt.figure(idx)    
    plt.show()

# **Using InceptionV3 Whith total finetuning**

In [24]:
# train all existing weights
for layer in inception.layers:
    layer.trainable = True

In [25]:
print("Building model with InceptionV3 with imagenet weights")
model_totalfinetuned = Sequential([
    inception,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(rate=0.2),
    Dense(38, activation='softmax')
])

# tell the model what cost and optimization method to use
model_totalfinetuned.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


model_totalfinetuned.summary()

Building model with InceptionV3 with imagenet weights
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten_2 (Flatten)         (None, 51200)             0         
                                                                 
 dense_4 (Dense)             (None, 512)               26214912  
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 38)                19494     
                                                                 
Total params: 48037190 (183.25 MB)
Trainable params: 48002758 (183.12 MB)
Non-trainable params: 34432 (134.50 KB)
__________________

In [26]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import pickle

filepath = "model_totalfinetuned_{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="acc", mode="max", patience=15)

callbacks_list = [checkpoint1, early] #early

history1 = model_totalfinetuned.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set), 
    callbacks=callbacks_list
)

with open('trainHistory1Dict.txt', 'wb') as file_pi:
    pickle.dump(history1.history, file_pi)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\jiten\AppData\Local\Temp\ipykernel_11168\846916163.py", line 11, in <module>
    training_set,
NameError: name 'training_set' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1428, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1319, in structured_traceback
    return VerboseTB.structur

In [27]:
acc = history1.history['accuracy']
val_acc = history1.history['val_accuracy']

loss = history1.history['loss']
val_loss = history1.history['val_loss']
EPOCHS=10

epochs_range = range(EPOCHS)

plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\jiten\AppData\Local\Temp\ipykernel_11168\2934163140.py", line 1, in <module>
    acc = history1.history['accuracy']
NameError: name 'history1' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1428, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\jiten\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1319, in structured_traceback
    return V

# Conslusion
model InceptionV3 without finetuning-->89%

model InceptionV3 2 top Blocks finetuning-->98%

model InceptionV3 totally finetuned-->96%